In [1]:
import matplotlib.pyplot as plt
from pyspark.sql.functions import *

# import pandas as pd
# df = spark.read.csv("/user/netID/hw8/cleaned_input.csv")
df = spark.read.csv("/Users/katelynwang/cs476file/final_project/hw8/cleaned_input.csv")


In [2]:
df = df.toDF(*["key","date","offense","ageGroup","sex","race","borough","longitude_latitude"])


In [3]:
# data profiling
df.summary().show()

+-------+-------------------+------------+--------------------+------------------+-------+--------------------+-------------+--------------------+
|summary|                key|        date|             offense|          ageGroup|    sex|                race|      borough|  longitude_latitude|
+-------+-------------------+------------+--------------------+------------------+-------+--------------------+-------------+--------------------+
|  count|            5116599|     5116599|             5114766|           5032691|5040014|             4741341|      4555618|             5116597|
|   mean|8.166756334949538E7|        null|  1152743.5359655172| 716.7056603773585|   null|                null|         null|                null|
| stddev|4.597853080077231E7|        null| 1.957388598561124E7|275.73680804435423|   null|                null|         null|                null|
|    min|           10025039|  01/01/2006|"DESTINATION; REF...|              1000|  BRONX|AMERICAN INDIAN/A...|       

In [4]:
df = df.filter(df.borough!='null')
df.show()

+---------+----------+--------------------+--------+---+-------+---------+--------------------+
|      key|      date|             offense|ageGroup|sex|   race|  borough|  longitude_latitude|
+---------+----------+--------------------+--------+---+-------+---------+--------------------+
| 10026695|01/27/2007| SMOKING IN ELEVATOR|   25-44|  F|UNKNOWN|   QUEENS|POINT (-73.932564...|
|109143320|03/22/2014|    OTHER ADMIN CODE|    null|  D|UNKNOWN|   QUEENS|POINT (-73.938627...|
|109143321|03/22/2014|    OTHER ADMIN CODE|    null|  D|UNKNOWN| BROOKLYN|POINT (-73.926928...|
|109143322|03/22/2014|    OTHER ADMIN CODE|    null|  D|UNKNOWN|   QUEENS|POINT (-73.831973...|
|109143324|03/22/2014|DISCON: CREATE HA...|   25-44|  M|UNKNOWN|    BRONX|POINT (-73.919888...|
|109143325|03/22/2014|    OTHER ADMIN CODE|   45-64|  M|UNKNOWN|    BRONX|POINT (-73.903177...|
|109143326|03/22/2014|CONSUMPTION OF AL...|   25-44|  M|UNKNOWN| BROOKLYN|POINT (-73.957726...|
|109143327|03/22/2014|CONSUMPTION OF AL.

In [5]:
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()

+---+----+-------+--------+-----+------+-------+------------------+
|key|date|offense|ageGroup|  sex|  race|borough|longitude_latitude|
+---+----+-------+--------+-----+------+-------+------------------+
|  0|   0|   1769|   79131|76487|370865|      0|                 0|
+---+----+-------+--------+-----+------+-------+------------------+



In [23]:
df = df.dropna()
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()

+---+----+-------+--------+---+----+-------+----+-----+
|key|date|offense|ageGroup|sex|race|borough|year|month|
+---+----+-------+--------+---+----+-------+----+-----+
|  0|   0|      0|       0|  0|   0|      0|   0|    0|
+---+----+-------+--------+---+----+-------+----+-----+



In [43]:
df.select('race').distinct().show()
df.select('ageGroup').distinct().show()
df.select('sex').distinct().show()
df.select('offense').distinct().show()
df.select('offense').distinct().count()

+--------------------+
|                race|
+--------------------+
|               BLACK|
|      WHITE HISPANIC|
|             UNKNOWN|
|               WHITE|
|AMERICAN INDIAN/A...|
|      BLACK HISPANIC|
|               OTHER|
|ASIAN / PACIFIC I...|
+--------------------+

+--------+
|ageGroup|
+--------+
|   25-44|
| UNKNOWN|
|   18-24|
|     <18|
|     65+|
|   45-64|
+--------+

+---+
|sex|
+---+
|  M|
|  F|
|  U|
+---+

+--------------------+
|             offense|
+--------------------+
|    RECKLESS DRIVING|
|      POSS. OF KNIFE|
|      ACCEPT ON HAIL|
|EVICTION; UNLAWFU...|
|DISCON: OBSTRUCT ...|
|OPERATION WHILE R...|
|NO EXIT SIGN AT D...|
|MEDALLION PLATE; ...|
|UNLICENSED TICKET...|
|UNREASONABLE NOIS...|
|    SMOKING ON A BUS|
|        BIKE IN PARK|
|REFUSE INSPECTION...|
|OTHER NAVIGATION LAW|
|IN A PUBLIC PLACE...|
|           FIREWORKS|
|TRIP SHEET; WITH ...|
|PEDICAB WITHOUT O...|
|BENCH; OBSTRUCT U...|
|UNSOLICITED RESPO...|
+--------------------+
only showing top 

569

In [6]:
df.dtypes

[('key', 'string'),
 ('date', 'string'),
 ('offense', 'string'),
 ('ageGroup', 'string'),
 ('sex', 'string'),
 ('race', 'string'),
 ('borough', 'string'),
 ('longitude_latitude', 'string')]

In [7]:
from pyspark.sql.functions import *
dates = df.select(col("date"),to_date(col("date"),"MM/dd/yyyy")).alias("date").show()

+----------+-------------------------+
|      date|to_date(date, MM/dd/yyyy)|
+----------+-------------------------+
|01/27/2007|               2007-01-27|
|03/22/2014|               2014-03-22|
|03/22/2014|               2014-03-22|
|03/22/2014|               2014-03-22|
|03/22/2014|               2014-03-22|
|03/22/2014|               2014-03-22|
|03/22/2014|               2014-03-22|
|03/22/2014|               2014-03-22|
|03/22/2014|               2014-03-22|
|03/23/2014|               2014-03-23|
|03/23/2014|               2014-03-23|
|03/22/2014|               2014-03-22|
|03/15/2014|               2014-03-15|
|03/22/2014|               2014-03-22|
|03/14/2014|               2014-03-14|
|03/22/2014|               2014-03-22|
|03/22/2014|               2014-03-22|
|03/21/2014|               2014-03-21|
|03/21/2014|               2014-03-21|
|03/22/2014|               2014-03-22|
+----------+-------------------------+
only showing top 20 rows



In [8]:
df = df.withColumn('date',to_date(col("date"),"MM/dd/yyyy"))
df = df.withColumn('year',date_format('date','yyyy'))


In [9]:
df = df.withColumn('month',date_format('date','M'))


In [10]:
df = df.filter((df.year=='2018')|(df.year=='2019'))
df = df.filter(col("borough").isNotNull())

In [11]:
df = df.replace('NEW YORK','MANHATTAN')

In [12]:
df = df.drop('longitude_latitude')
df.groupBy("borough").count().show()

+-------------+-----+
|      borough|count|
+-------------+-----+
|     BROOKLYN|34363|
|       QUEENS|22592|
|STATEN ISLAND| 5378|
|        BRONX|32129|
|    MANHATTAN|44531|
+-------------+-----+



In [19]:
exp_df = df.groupBy("borough","date").count()

In [20]:
exp_df.show()

+-------------+----------+-----+
|      borough|      date|count|
+-------------+----------+-----+
|     BROOKLYN|2018-01-27|   77|
|     BROOKLYN|2018-09-20|   39|
|       QUEENS|2018-07-21|   18|
|     BROOKLYN|2018-08-04|   38|
|STATEN ISLAND|2018-06-13|    6|
|       QUEENS|2019-01-10|   30|
|    MANHATTAN|2018-05-06|   30|
|     BROOKLYN|2018-01-29|   35|
|       QUEENS|2018-03-08|   34|
|     BROOKLYN|2018-03-14|   43|
|    MANHATTAN|2018-03-25|   21|
|STATEN ISLAND|2018-04-05|    8|
|STATEN ISLAND|2018-04-28|   19|
|STATEN ISLAND|2018-05-24|    5|
|STATEN ISLAND|2018-06-27|    9|
|        BRONX|2018-09-27|   24|
|     BROOKLYN|2018-12-03|   22|
|    MANHATTAN|2018-12-06|   57|
|       QUEENS|2018-12-17|    5|
|     BROOKLYN|2019-03-16|   39|
+-------------+----------+-----+
only showing top 20 rows



In [21]:
df.groupBy("borough","date").count().show()

+-------------+----------+-----+
|      borough|      date|count|
+-------------+----------+-----+
|     BROOKLYN|2018-01-27|   77|
|     BROOKLYN|2018-09-20|   39|
|       QUEENS|2018-07-21|   18|
|     BROOKLYN|2018-08-04|   38|
|STATEN ISLAND|2018-06-13|    6|
|       QUEENS|2019-01-10|   30|
|    MANHATTAN|2018-05-06|   30|
|     BROOKLYN|2018-01-29|   35|
|       QUEENS|2018-03-08|   34|
|     BROOKLYN|2018-03-14|   43|
|    MANHATTAN|2018-03-25|   21|
|STATEN ISLAND|2018-04-05|    8|
|STATEN ISLAND|2018-04-28|   19|
|STATEN ISLAND|2018-05-24|    5|
|STATEN ISLAND|2018-06-27|    9|
|        BRONX|2018-09-27|   24|
|     BROOKLYN|2018-12-03|   22|
|    MANHATTAN|2018-12-06|   57|
|       QUEENS|2018-12-17|    5|
|     BROOKLYN|2019-03-16|   39|
+-------------+----------+-----+
only showing top 20 rows



In [34]:
df.groupBy("year","month").count().show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2018|    1| 5346|
|2018|    9| 3599|
|2018|    6| 3876|
|2018|    8| 4197|
|2018|    7| 4582|
|2019|    1| 3310|
|2018|    3| 4970|
|2019|    4| 2825|
|2018|   10| 3421|
|2019|   11| 2249|
|2019|    6| 3101|
|2019|    8| 3014|
|2019|    3| 3310|
|2018|   12| 2883|
|2019|   12| 1771|
|2019|   10| 2187|
|2018|   11| 3032|
|2018|    5| 4567|
|2019|    7| 3692|
|2019|    9| 2849|
+----+-----+-----+
only showing top 20 rows



In [29]:
# MOST COMMON OFFENSE
df.groupBy("offense").count().sort(col('count').desc()).show()

+--------------------+-----+
|             offense|count|
+--------------------+-----+
|ALCOHOLIC BEVERAG...| 5678|
|OPERATION WHILE R...| 5279|
|NON PAYMENT OF FA...| 4559|
|            TRESPASS| 4541|
|    RECKLESS DRIVING| 4017|
|CONSUMPTION OF AL...| 3432|
|OBSTRUCTS VEHICUL...| 3406|
|MOTOR VEHICLE; EN...| 3261|
|      ACCEPT ON HAIL| 3041|
|FEDERAL MOTOR VEH...| 2591|
|  DISORDERLY CONDUCT| 2245|
|UNLICENSED VEHICL...| 1541|
|   OTHER TRANSIT REG| 1323|
|CREATES A HAZARDO...| 1221|
|REVOKED REGISTRATION| 1115|
|    PUBLIC URINATION| 1105|
|CONGREGATES WITH ...| 1041|
|PICK UP FROM BUS ...|  956|
|UNLAWFUL POSSESSI...|  861|
|           OTHER ABC|  828|
+--------------------+-----+
only showing top 20 rows



In [33]:
# BY GENDER
df.filter(df.sex == 'M').groupBy("offense").count().sort(col('count').desc()).show()
df.filter(df.sex == 'F').groupBy("offense").count().sort(col('count').desc()).show()

+--------------------+-----+
|             offense|count|
+--------------------+-----+
|ALCOHOLIC BEVERAG...| 5169|
|OPERATION WHILE R...| 4141|
|NON PAYMENT OF FA...| 4039|
|    RECKLESS DRIVING| 3601|
|            TRESPASS| 3547|
|CONSUMPTION OF AL...| 3153|
|      ACCEPT ON HAIL| 2978|
|OBSTRUCTS VEHICUL...| 2949|
|MOTOR VEHICLE; EN...| 2917|
|FEDERAL MOTOR VEH...| 2055|
|  DISORDERLY CONDUCT| 1778|
|UNLICENSED VEHICL...| 1498|
|   OTHER TRANSIT REG| 1220|
|    PUBLIC URINATION| 1053|
|CREATES A HAZARDO...| 1049|
|PICK UP FROM BUS ...|  948|
|REVOKED REGISTRATION|  852|
|CONGREGATES WITH ...|  811|
|UNLAWFUL POSSESSI...|  797|
|RESTRICTIONS ON T...|  715|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
|             offense|count|
+--------------------+-----+
|OPERATION WHILE R...| 1134|
|            TRESPASS|  981|
|ALCOHOLIC BEVERAG...|  503|
|NON PAYMENT OF FA...|  502|
|OBSTRUCTS VEHICUL...|  452|
|  DISORDERLY CONDUCT|  451|
|    RECKLESS DRI

In [36]:
# WAHT AGEGROUP HAS THE HIGHEST NUMBER OF SUMMON, IN WHICH AREA?
df.groupBy("ageGroup","borough").count().sort(col('count').desc()).show()

+--------+-------------+-----+
|ageGroup|      borough|count|
+--------+-------------+-----+
|   25-44|        BRONX|12525|
|   25-44|     BROOKLYN|11344|
|   25-44|    MANHATTAN|10538|
|   25-44|       QUEENS| 6990|
|   45-64|     BROOKLYN| 6787|
|   45-64|        BRONX| 5555|
|   18-24|        BRONX| 5198|
|   45-64|    MANHATTAN| 4750|
|   45-64|       QUEENS| 3804|
|   18-24|     BROOKLYN| 3716|
|   18-24|    MANHATTAN| 3380|
|   18-24|       QUEENS| 2290|
|   25-44|STATEN ISLAND| 1675|
|   45-64|STATEN ISLAND|  804|
|     <18|     BROOKLYN|  759|
|     <18|        BRONX|  746|
|   18-24|STATEN ISLAND|  636|
|     65+|     BROOKLYN|  613|
|     65+|       QUEENS|  376|
|     65+|    MANHATTAN|  354|
+--------+-------------+-----+
only showing top 20 rows



In [44]:
# RACE
df.groupBy("race","borough").count().sort(col('count').desc()).show()
df.groupBy("race","offense").count().sort(col('count').desc()).show()
df.groupBy("race","ageGroup").count().sort(col('count').desc()).show()

+--------------------+-------------+-----+
|                race|      borough|count|
+--------------------+-------------+-----+
|               BLACK|     BROOKLYN|12952|
|               BLACK|        BRONX|10630|
|      WHITE HISPANIC|        BRONX| 7522|
|               BLACK|       QUEENS| 6331|
|               BLACK|    MANHATTAN| 6219|
|               WHITE|    MANHATTAN| 4527|
|      BLACK HISPANIC|        BRONX| 4339|
|      WHITE HISPANIC|     BROOKLYN| 3934|
|      WHITE HISPANIC|       QUEENS| 3509|
|      WHITE HISPANIC|    MANHATTAN| 3465|
|               WHITE|     BROOKLYN| 2654|
|ASIAN / PACIFIC I...|       QUEENS| 2031|
|ASIAN / PACIFIC I...|    MANHATTAN| 1654|
|             UNKNOWN|    MANHATTAN| 1539|
|             UNKNOWN|     BROOKLYN| 1478|
|      BLACK HISPANIC|    MANHATTAN| 1403|
|               BLACK|STATEN ISLAND| 1354|
|ASIAN / PACIFIC I...|     BROOKLYN| 1286|
|               WHITE|       QUEENS|  994|
|      WHITE HISPANIC|STATEN ISLAND|  861|
+----------

In [45]:
df.groupBy("race","ageGroup","borough").count().sort(col('count').desc()).show()

+--------------+--------+---------+-----+
|          race|ageGroup|  borough|count|
+--------------+--------+---------+-----+
|         BLACK|   25-44| BROOKLYN| 6164|
|         BLACK|   25-44|    BRONX| 5443|
|WHITE HISPANIC|   25-44|    BRONX| 3918|
|         BLACK|   45-64| BROOKLYN| 3900|
|         BLACK|   25-44|   QUEENS| 3237|
|         BLACK|   25-44|MANHATTAN| 3097|
|         WHITE|   25-44|MANHATTAN| 2709|
|         BLACK|   45-64|    BRONX| 2487|
|BLACK HISPANIC|   25-44|    BRONX| 2192|
|         BLACK|   18-24|    BRONX| 2179|
|         BLACK|   18-24| BROOKLYN| 2019|
|WHITE HISPANIC|   25-44| BROOKLYN| 1986|
|WHITE HISPANIC|   25-44|   QUEENS| 1898|
|WHITE HISPANIC|   25-44|MANHATTAN| 1878|
|         BLACK|   45-64|MANHATTAN| 1822|
|         BLACK|   45-64|   QUEENS| 1724|
|WHITE HISPANIC|   45-64|    BRONX| 1711|
|WHITE HISPANIC|   18-24|    BRONX| 1571|
|         WHITE|   25-44| BROOKLYN| 1316|
|BLACK HISPANIC|   18-24|    BRONX| 1161|
+--------------+--------+---------